In [7]:
#Alur
#Setup
#Buka File
#Lihat data
#Tentukan X dan y
#Cek missing value
#Handle missing value
#Feature Engineering
#Buat Model
#Cari RMSE paling kecil
#fit model ke data test
#Finishing

In [20]:
#Setup
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

print("Setup Done")

Setup Done


In [47]:
#Import Data
data = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/train.csv")
tdata = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/test.csv")
print(data.shape)

(630000, 13)


In [ ]:
#Lihat data
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [14]:
mis = data.isnull().sum()
mis[0:13]

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64

In [74]:
#cari kolom kategorikal untuk model catboost

for col in data:
    if data[col].dtype == "object":
        print(col)

cat_col= [
    'course',
    'sleep_quality',
    'study_method',
    'facility_rating',
    'gender',
    'internet_access',
    'exam_difficulty'
]

gender
course
internet_access
sleep_quality
study_method
facility_rating
exam_difficulty


In [77]:
#tentukan X dan y
y = data["exam_score"]
X = data.drop(["id","exam_score"], axis = 1)
# X = data.drop(["id","exam_score","internet_access","gender","exam_difficulty"], axis = 1)

In [78]:
#buat model
model = CatBoostRegressor(
    iterations=1500, 
    learning_rate= 0.05,
    depth = 6,
    loss_function= 'RMSE',
    cat_features= cat_col,
    random_seed= 2)

In [79]:
#Train model
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size= 0.2, random_state=1)
model.fit(train_X,train_y)

0:	learn: 18.3134419	total: 348ms	remaining: 8m 42s
1:	learn: 17.7379292	total: 647ms	remaining: 8m 4s
2:	learn: 17.1984954	total: 912ms	remaining: 7m 35s
3:	learn: 16.6914073	total: 1.14s	remaining: 7m 7s
4:	learn: 16.2211739	total: 1.39s	remaining: 6m 54s
5:	learn: 15.7777506	total: 1.59s	remaining: 6m 36s
6:	learn: 15.3609647	total: 1.79s	remaining: 6m 21s
7:	learn: 14.9746149	total: 2.05s	remaining: 6m 22s
8:	learn: 14.6109102	total: 2.23s	remaining: 6m 9s
9:	learn: 14.2722201	total: 2.45s	remaining: 6m 5s
10:	learn: 13.9394652	total: 2.72s	remaining: 6m 7s
11:	learn: 13.6325935	total: 2.9s	remaining: 5m 59s
12:	learn: 13.3468081	total: 3.11s	remaining: 5m 56s
13:	learn: 13.0831932	total: 3.36s	remaining: 5m 56s
14:	learn: 12.8279640	total: 3.57s	remaining: 5m 53s
15:	learn: 12.5937176	total: 3.76s	remaining: 5m 49s
16:	learn: 12.3546476	total: 3.99s	remaining: 5m 48s
17:	learn: 12.1339403	total: 4.23s	remaining: 5m 48s
18:	learn: 11.9184884	total: 4.42s	remaining: 5m 44s
19:	learn

In [80]:
#predict
preds_vy= model.predict(val_X) #predict jawaban ujian
preds_ty= model.predict(train_X) #predict jawaban latihan soal

In [81]:
#Model Validation
train_rmse = root_mean_squared_error(train_y, preds_ty)
val_rmse = root_mean_squared_error(val_y,preds_vy)
print(train_rmse)
print(val_rmse)

8.73604298553151
8.774883378731257


In [76]:
penting = model.get_feature_importance(prettified=True)
print(penting)

          Feature Id  Importances
0        study_hours    51.183935
1   class_attendance    19.618945
2      sleep_quality     9.709697
3       study_method     8.267085
4    facility_rating     6.664085
5        sleep_hours     4.311290
6                age     0.099738
7             course     0.087366
8    exam_difficulty     0.024533
9             gender     0.022903
10   internet_access     0.010425


In [ ]:
#fit model ke Semua data
model.fit(X,y)
